In [1]:
import pandas as pd
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [43]:
# gets top 200 movies for every genre 

movie_list = []

response = requests.get('http://www.imdb.com/chart/top?ref_=nv_mv')
HTML = response.text
genres = Selector(text=HTML).xpath("//ul[@class='quicklinks ']/li[@class='subnav_item_main']/a/@href").extract()

for link in genres:
    
    response = requests.get('http://www.imdb.com'+link)
    HTML = response.text
    movies = Selector(text=HTML).xpath("//h3[@class='lister-item-header']/a/@href").extract()
    
    response = requests.get('http://www.imdb.com'+link.replace("ref_=chttp_gnr_1",'&page=2&ref_=adv_nxt'))
    HTML = response.text
    movies2 = Selector(text=HTML).xpath("//h3[@class='lister-item-header']/a/@href").extract()
    
    response = requests.get('http://www.imdb.com'+link.replace("ref_=chttp_gnr_1",'&page=3&ref_=adv_nxt'))
    HTML = response.text
    movies3 = Selector(text=HTML).xpath("//h3[@class='lister-item-header']/a/@href").extract()
    
    response = requests.get('http://www.imdb.com'+link.replace("ref_=chttp_gnr_1",'&page=4&ref_=adv_nxt'))
    HTML = response.text
    movies4 = Selector(text=HTML).xpath("//h3[@class='lister-item-header']/a/@href").extract()
    
    movie_list = list(set(movie_list + movies + movies2 + movies3 + movies4))

In [44]:
# goes through movie list and collects data from the movies page 
# appends movie data to movie_data dataframe 

movie_data = pd.DataFrame(columns = ['Link','Title','Score','Reviews','Plot_Summary',"Director","Writers","Stars",
                                    "Storyline",'Genres','Budget', "Date"])

for i, movie in enumerate(movie_list):
    df = pd.DataFrame()
    
    response = requests.get('http://www.imdb.com' + movie)
    HTML = response.text
    
    movie_link = 'http://www.imdb.com' + movie
    title = Selector(text=HTML).xpath("//div[@class='titleBar']/div[@class='title_wrapper']/h1/text()").extract()[0]
    score = Selector(text=HTML).xpath("//strong/span/text()").extract()[0]
    reviews = Selector(text=HTML).xpath("//div[@class='imdbRating']/a/span[@class='small']/text()").extract()[0]
    plot_summary = Selector(text=HTML).xpath("//div[@class='summary_text']/text()").extract()[0].replace("\n",'')
    director = Selector(text=HTML).xpath("//div[@class='credit_summary_item'][1]/span/a/span[@class='itemprop']/text()").extract()
    writers = Selector(text=HTML).xpath("//div[@class='credit_summary_item'][2]/span/a/span[@class='itemprop']/text()").extract()
    stars = Selector(text=HTML).xpath("//div[@class='credit_summary_item'][3]/span/a/span[@class='itemprop']/text()").extract()
    storyline = Selector(text=HTML).xpath("//div[@class='inline canwrap']/p/text()").extract()[0].replace("\n",'')
    genres = Selector(text=HTML).xpath("//div[@class='see-more inline canwrap'][2]/a/text()").extract()
    budget = Selector(text=HTML).xpath("//div[@class='txt-block'][7]/text()").extract()[1].replace("\n",'')
    date = Selector(text=HTML).xpath("//div[@class='titleBar']/div[@class='title_wrapper']/div[@class='subtext']/a/text()").extract()[0].replace("\n",'').split(" (")[0]
    
    movie_data = movie_data.append({'Link':movie_link, "Title": title, "Score": score, 
                                    "Reviews": reviews, 'Plot_Summary': plot_summary,
                                   "Director": director, "Writers":writers, "Stars":stars,
                                   "Storyline":storyline, "Genres":genres, "Budget": budget,
                                   "Date": date}, ignore_index = True)
    
    if i%10 == 0:
        sys.stdout.write('\r'+ str((float(i)/len(movie_list))*100)+'%')

In [330]:
# movie_data.to_excel('/Users/caitlin.mowdy/Desktop/movies3.json')